### Importing libraries

In [1]:
import llama_index.core
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_parse import LlamaParse
import nest_asyncio; nest_asyncio.apply()

import json
import os
import re
from dotenv import load_dotenv
load_dotenv()

True

### Loading data and index

In [2]:
BP_threshold = 30
top_k = 24 # set how many chunks are given as context to GPT

PERSIST_DIR = f"./storage_SDR_semantic_chunking_various_thresholds/storage_SDR_sem_{BP_threshold}_text-embedding-ada-002"

storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
index = load_index_from_storage(storage_context)

LIST_OF_DOCS = ["raw_texts/"+f for f in os.listdir("Raw_texts")]
print(LIST_OF_DOCS)

# Open quizzes
LIST_OF_QUIZZES = ["JSON_quizzes_only_closed_questions/"+f for f in os.listdir("./JSON_quizzes_only_closed_questions") if f.endswith('.json')]
print(LIST_OF_QUIZZES)

['raw_texts/2-settext.pdf', 'raw_texts/3-settext.txt', 'raw_texts/3-slides.pdf', 'raw_texts/4-settext.pdf', 'raw_texts/4-slides.pdf', 'raw_texts/5-settext.pdf', 'raw_texts/9-settext.pdf']
['JSON_quizzes_only_closed_questions/itembank-2.json', 'JSON_quizzes_only_closed_questions/itembank-3-4.json', 'JSON_quizzes_only_closed_questions/itembank-5-6.json', 'JSON_quizzes_only_closed_questions/itembank-9a.json', 'JSON_quizzes_only_closed_questions/itembank-9b.json']


###  RAG

In [3]:
model = "gpt-4o-mini"

Settings.llm = OpenAI(temperature=0.0, model=model)

def pingGPT(text_input):
   response = query_engine.query(text_input)
   return response.response

### The retriever with default prompt

In [4]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

# configure response synthesizer

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

from llama_index.core import PromptTemplate


### Full loop

In [ ]:
total_nquestions = 0
total_ncorrects = 0
itembank_reports = []

itembank_reports.append(f"breakpoint percentile: {BP_threshold}")
itembank_reports.append(f"top_k: {top_k}")

# Iterate over each question and get GPT's response
for doc in LIST_OF_QUIZZES:
    with open(doc, 'r', encoding='UTF-8') as file:
        questions_JSON = json.load(file)
    
    itembank_number = re.search(r"\d[a|b]?(?:\-\d)?", doc).group()
    this_ncorrects = 0 # for correct answers in each separate itembank
    this_nquestions = 0

    print(f"----------processing itembank-{itembank_number}----------")

    for question in questions_JSON:
        full_prompt = question['question'] + '\n' + '\n'.join(question['answers'])
        given_answer = pingGPT(full_prompt)  # Get the response from GPT
        question["GPT's response"] = given_answer  # Append the response to the question
        question["correct"] = given_answer==question["correct answer(s)"]

        this_nquestions+=1
        total_nquestions+=1
        if given_answer==question["correct answer(s)"]:
            total_ncorrects+=1
            this_ncorrects+=1
        
    itembank_reports.append(f"itembank-{itembank_number}:\t\tCorrect answers {this_ncorrects} / total questions {this_nquestions} * 100 = {round(this_ncorrects/this_nquestions*100, 2)}%")


    # Save the updated JSON data to a new file
    with open(f'./results/improved_RAG_responses/semantic_chunking/RAG_{model}_semantic_{BP_threshold}_responses-{itembank_number}.json', 'w') as outfile:
        json.dump(questions_JSON, outfile, indent=2)

itembank_reports.append(f"\nOverall correct answers {total_ncorrects} / total questions {total_nquestions} * 100 = {round(total_ncorrects/total_nquestions*100, 2)}%\n")
itembank_reports.append("------------------------------------------------------------------------\n\n")

# save the prompt and number of correct questions
with open(f"./results/default_RAG_{model}_semantic_chunking_results.txt", 'a', encoding='UTF-8') as report_file:
    report_file.write('\n'.join(itembank_reports))

----------processing itembank-2----------
----------processing itembank-3-4----------
----------processing itembank-5-6----------
----------processing itembank-9a----------
----------processing itembank-9b----------


: 

### Observe with Phoenix (not necessary to run)

In [ ]:
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={os.getenv('PHOENIX_API_KEY')}"
llama_index.core.set_global_handler("arize_phoenix", endpoint="https://llamatrace.com/v1/traces")

with open('./JSON_questions_only_closed_questions/itembank-3-4.json', 'r', encoding='UTF-8') as file:
    questions_JSON = json.load(file)

for question in questions_JSON[18:19]:
    full_prompt = question['question'] + '\n' + '\n'.join(question['answers'])
    given_answer = pingGPT(full_prompt)  # Get the response from GPT
print(given_answer)